Last updated on 7 August 2017

In [1]:
import numpy as np
import scipy.io as sio
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

# Regularized Linear Regression

In the first half of this exercise, we implement regularized linear regression to predict the amount of water flowing out of a dam using the change of water level in a reservoir.

The dataset has already been divided into three parts for our convenience, comprising of:

- A training set that your model will learn on: X, y
- A cross validation set for determining the regularization parameter:
Xval, yval
- A test set for evaluating performance. These are “unseen” examples
which your model did not see during training: Xtest, ytest

In [2]:
data = sio.loadmat("ex5data1.mat")
X = data["X"].flatten(); y = data["y"].flatten()
Xval = data["Xval"].flatten(); yval = data["yval"].flatten()
Xtest = data["Xtest"].flatten(); ytest = data["ytest"].flatten()

The data are column vectors shaped as 12 by 1 two dimensional numpy arrays, so it is necessary to flatten them to a 1D numpy array to facilitate easy plotting and subsequent numerical computation.

## Visualizing the dataset

We visualize all three sets in the same figure. To view a set in isolation, double click on the name of that particular set in the legend at the top right hand corner, and the other two sets will be temporarily removed from the figure. To bring them back, double click on that same set's name again.

In [3]:
trgdat = go.Scatter(x=X, y=y, mode="markers", name = "Training")
valdat = go.Scatter(x=Xval, y=yval, mode="markers", name = "Cross Validation")
testdat = go.Scatter(x=Xtest, y=ytest, mode="markers", name = "Test")
plotly.offline.iplot({"data": [trgdat,valdat,testdat],
                      "layout": go.Layout(title = "Scatter plot of training data",
                                          xaxis = dict(title="Change in water level (x)"),
                                          yaxis = dict(title="Water flowing out of the dam (y)"))})

## Regularized linear regression cost function

In [4]:
def linear_regression_cost(_theta, _X, _y, _lda):
    regterm = _lda/(2*_y.size) * np.sum(np.square(_theta))
    return 1/(2*_y.size) * np.sum(np.square(_X@_theta - _y)) + regterm

#### Implementation notes for `add_bias_term()` function

Function that accepts an input data matrix and returns as output a design matrix by adding a column of ones to the original data matrix..

Note if `_X` is 1D array, performing a transpose on it will not have any effect, but if `_X` is a 2D array (which is the case when `_X` holds polynomial features), then the transpose will take effect to prepare the array to be stacked vertically with the 1D array of ones. Finally a transpose is performed on the stacked 2D array to put it in the correct shape of a design matrix (to suit the linear algebra formulation of a regression).

TODO: replace the old implementation of this function in the previous exercises 3 and 4, with this new implementation that uses `np.vstack()` instead of `np.concatenate()`.

In [5]:
def add_bias_term(_X):
    ones_row = np.ones(_X.shape[0])
    _X_bias = np.vstack((ones_row, _X.T)).T
    return _X_bias

## Regularized linear regression gradient

TODO: vectorization here. explain how this line `(X_bias@Theta - y).reshape((len(y)),1) *X_bias` shows broadcasting in action! The important thing I've learnt about broadcasting is that it works based on matching axes....
by reshaping `X_bias@Theta - y` from shape `(12, )` to `(12, 1)`, I turned it into a column which could be broadcasted in the row (horizontal) direction of `X_bias` of shape `(12, 2)`, meaning the `*` operation could be applied to multiply the reshaped `(12,1)` vector to each column of X_bias on the left.
At the end, a simple numpy sum along axis 0 gives us the result we need

`np.sum((X_bias@Theta - y).reshape((len(y)),1) *X_bias,axis =0)`

In [6]:
def linear_regression_gradient(_theta,_X,_y,_lda):
    regterm = _lda/_y.size * _theta
    regterm[0] = 0 # do not regularize the zeroth term
    return 1/_y.size * np.sum((_X@_theta - _y).reshape((_y.size),1) * _X,axis =0) + regterm

## Fitting linear regression

Finally realized that the `scipy.optimize` library contains an `fmin_cg()` routine that directly interfaces with an implementation of the Polack-Ribiere flavor of conjugate gradient optimization, the same flavor of conjugate gradient that was given to us in the MATLAB `fmin_cg.m` file provided by the course. 

We are using `fmin_cg()` directly here instead of `minimize()` with the passed parameter `method = 'CG'`.

In [7]:
from scipy.optimize import fmin_cg

def train_linear_regression(_X, _y, _lda):
    _theta_optimized = fmin_cg(linear_regression_cost, np.zeros(_X.shape[1]),
                              linear_regression_gradient, (_X,_y,_lda), disp=True)
    return _theta_optimized

We train the parameters of our linear model without regularization. At such a low dimension, our hypothesis will not benefit much from regularization so we set $\lambda=0$.

In [8]:
theta_optimized = train_linear_regression(add_bias_term(X), y, 0)

Optimization terminated successfully.
         Current function value: 22.373906
         Iterations: 2
         Function evaluations: 5
         Gradient evaluations: 5


We compute and visualize our model, observing that it is a poor fit to the data since the data has a non-linear trend that will only be better predicted with a polynomial model, which we will implement later. Before training a polynomial model, we will visualize the learning curves of this linear model to learn how they should look like for such a model that underfits the data.

In [9]:
X_predict = np.linspace(X.min(),X.max())
X_pred_bias = add_bias_term(X_predict)
y_predict = X_pred_bias@theta_optimized

trgdat = go.Scatter(x=X, y=y, mode="markers", name = "Data")
line = go.Scatter(x=X_predict, y=y_predict, mode="line", name="Regression Line")
plotly.offline.iplot({"data": [trgdat,line],
                      "layout": go.Layout(title = "Linear Fit",
                                          xaxis = dict(title="Change in water level (x)"),
                                          yaxis = dict(title="Water flowing out of the dam (y)"))})

# Bias-Variance
An important concept in machine learning is the bias-variance tradeoff. Models with high bias are not complex enough for the data and tend to underfit, while models with high variance overfit to the training data. Aside from the course materials, an easy but informative read on this concept can be found [here](https://ml.berkeley.edu/blog/2017/07/13/tutorial-4/?imm_mid=0f493b&cmp=em-data-na-na-newsltr_ai_20170724).

## Learning Curves

We now implement code to generate the learning curves that will be useful in debugging learning algorithms. Recall that a learning curve plots training and cross validation error as a function of training set size.

#### Implementation notes for `learning_curve()` function

TODO: explain implementation note that I index out the ith values from the `_y` array (from training data) first and also wrap them up with np.asarray() because I need to use them in learning the parameters theta, meaning it will be used in cost and gradient functions.

This also forced me to convert my above cost and gradient to not using len(y) anymore, but instead use y.size, since for the first iteration, `np.asarray(_y[i])` would give me a scalar in an array at least still letting it be one element size array, there will be a size attribute which can return the size as 1 

In computing the cross val errors, we use the entire training dataset, why? see diary entry on 28 July Friday

Function requires the bias term to be added to inputs prior to passing into the function

In [10]:
def learning_curve(_X, _y, _Xval, _yval, _lda):
    # Initialize variables for computation inputs and storage
    _trainerr = np.zeros_like(_y)
    _cverr = np.zeros_like(_y)
    numparams = _X.shape[1] # get the number of columns in our design matrix be equal to number of parameter needed to learn
    # Select out subset of our training data, starting from just first data point to eventuall all the data points
    for i in range(0, _y.size):
        _yi = np.asarray(_y[:i+1]) 
        # Learn the parameters on the given subset of our training data, initialized to 0 prior to training      
        _thetalearnt = fmin_cg(linear_regression_cost, np.zeros(numparams),
                               linear_regression_gradient, (_X[:i+1], _yi, _lda),  
                               disp=False)
        # Compute the errors on the training and cross validation set
        # We never regularize these errors, and so we hardcode lambda as 0
        _trainerr[i] = linear_regression_cost(_thetalearnt,_X[:i+1], _yi, 0)
        _cverr[i] = linear_regression_cost(_thetalearnt, _Xval, _yval, 0)
    return _trainerr, _cverr

We observe that both the train error and cross validation error are high when the number of training examples is increased. This reflects a high bias problem in the model – the linear regression model is too simple and is unable to fit our dataset well.

In [11]:
trainerr, cverr = learning_curve(add_bias_term(X),y,add_bias_term(Xval),yval,0)

numtrngegs = list(range(1,13))
trainlearncurve = go.Scatter(x=numtrngegs, y=trainerr, mode="line", name = "Train")
cvlearncurve = go.Scatter(x=numtrngegs, y=cverr, mode="line", name="Cross Validation")
plotly.offline.iplot({"data": [trainlearncurve,cvlearncurve],
                      "layout": go.Layout(title = "Learning curve for linear regression",
                                          xaxis = dict(title="Number of training examples used to learn the parameters of our model"),
                                          yaxis = dict(title="Error"))})

# Polynomial Regression

We now overcome the initial problem of underfitting by a linear model, by adding more features, and hence more parameters to learn, with a polynomial model. We will add more features by using the higher powers of the one and only existing feature in the dataset.

In [12]:
def poly_features(_X, degree):
    polyfeats = map(lambda pwr : _X**pwr, range(1,degree+1)) # iterable object of power raised arrays
    _Xpoly = np.vstack(polyfeats).T  #Perform a transpose to get the matrix into our desired dimension m by p
    return _Xpoly

#### Implementation notes for `feature_normalize()` function

The MATLAB skeleton code implementing feature normalization as given in this course, is similar to an example in the [MATLAB documentation for `bsxfun()`](https://www.mathworks.com/help/matlab/ref/bsxfun.html), and looks like:
```
mu = mean(X);
X_norm = bsxfun(@minus, X, mu);
sigma = std(X_norm);
X_norm = bsxfun(@rdivide, X_norm, sigma);
```
All the MATLAB functions were used in their default configuration. In order for our python implementation to be consistent, we have to do the following:

- Mean and standard deviation for MATLAB matrices (2D arrays) are computed column-wise returning vectors by default, whereas in Numpy, they are computed globally for all elements in the flattened array, returning scalars. Passing in `axis=0` ensures that we replicate the MATLAB behavior in Numpy.

- In MATLAB, standard deviation is normalized by $N-1$ by default (i.e. degrees of freedom is 1), where $N$ is the number of observations. In Numpy, standard deviation is normalized by $N$ by default (i.e. degrees of freedom is 0), so passing in `ddof=1` ensures that we match the default MATLAB behavior.

Broadcasting happens automatically in Numpy based on the dimensions of the arrays, without the need to call a function, unlike in MATLAB where it is necessary to call `bsxfun()`.

TODO: This also reminds me that in programming exercise 1, where we need to do feature normalization, I also have a big issue to fix..... just calling np.mean() on an array does not mean its a vectorized mean, but a global mean of all the elements in the array...so I gotta go back to fix it.....

In [13]:
def feature_normalize(_X):
    _mu = np.mean(_X, axis=0)
    _Xnorm = _X-_mu
    _sigma = np.std(_Xnorm, axis=0, ddof=1)
    _Xnorm = _Xnorm / _sigma
    return _Xnorm, _mu, _sigma

## Learning Polynomial Regression

We will train a polynomial of degree 8. Feature normalization is necessary because high polynomial powers of the the higher order features that we are generating give rise to large values that would make training problematic.

In [14]:
deg = 8
Xpoly,mu,sigma = feature_normalize(poly_features(X,deg))
#Qn why do we use the mu and sigma from the training data to normalize the cross validation and test datasets?
# Compute feature scaling for cross validation data
Xvalpoly =poly_features(Xval,deg)
Xvalpoly = Xvalpoly-mu
Xvalpoly = Xvalpoly / sigma

In [15]:
lda_init = 0
theta_optimized = train_linear_regression(add_bias_term(Xpoly), y, lda_init)

         Current function value: 0.145323
         Iterations: 66
         Function evaluations: 142
         Gradient evaluations: 130


In order to visualize our model with the parameters we have learnt, we need to also normalize the features of our artificial data set that "spans" the entire data space, similar to in Programming Exercise 1. The predictions of the model using this "spanning" dataset as input would be directly map to the original output scale and do not require any normalization.

In [16]:
span = np.linspace(X.min()-10,X.max()+10)  # Add additional spaces at the ends to demonstrate the over fitting
# spanpoly is just a defined range of X values to scale for plotting a curve
# Normalize the features in spanpoly before making predictions
spanpoly = poly_features(span,deg)
spanpoly = spanpoly-mu
spanpoly = spanpoly / sigma
yspan = add_bias_term(spanpoly)@theta_optimized
#print(ypoly_predict)
trgdat = go.Scatter(x=X, y=y, mode="markers", name = "Data")
line = go.Scatter(x=span, y=yspan, mode="line", name="Regression Curve")
plotly.offline.iplot({"data": [trgdat,line],
                      "layout": go.Layout(title = "Polynomial Fit",
                                          xaxis = dict(title="Change in water level (x)"),
                                          yaxis = dict(title="Water flowing out of the dam (y)"))})

We see that the polynomial fit is able to follow the datapoints very well - thus, obtaining a low training error. However, the polynomial fit is very complex and even drops off at the extremes. This is an indicator that the polynomial regression model is overfitting the training data and will not generalize well.

This is further verified by the learning curves below where the training error is low, but the cross validation error is high. There is a gap between the training and cross validation errors, indicating a problem of high variance.

TODO: The trends match those of figure 5 in ex5.pdf, but the actual values of the points on these curves do not. Find out why...

In [17]:
polytrainerr, polycverr = learning_curve(add_bias_term(Xpoly),y,add_bias_term(Xvalpoly),yval,lda_init)

numtrngegs = list(range(1,13))
polytrainlearncurve = go.Scatter(x=numtrngegs, y=polytrainerr, mode="line", name = "Train")
polycvlearncurve = go.Scatter(x=numtrngegs, y=polycverr, mode="line", name="Cross Validation")
plotly.offline.iplot({"data": [polytrainlearncurve,polycvlearncurve],
                      "layout": go.Layout(title = "Learning curve for polynomial regression",
                                          xaxis = dict(title="Number of training examples used to learn the parameters of our model"),
                                          yaxis = dict(title="Error"))})

## Adjusting the Regularization Parameter $\lambda$

Let us explore how different values for the regularization parameter affects the quality of our polynomial fittings, and their corresponding learning curves. The results speak for themselves.

### For $\lambda = 1$, the model is good. 

It does not suffer from high bias or high variance, effectively achieving a good trade-off between bias and variance.


In [18]:
lda_init = 1
theta_optimized = train_linear_regression(add_bias_term(Xpoly), y, lda_init)
yspan = add_bias_term(spanpoly)@theta_optimized
trgdat = go.Scatter(x=X, y=y, mode="markers", name = "Data")
line = go.Scatter(x=span, y=yspan, mode="line", name="Regression Curve")
plotly.offline.iplot({"data": [trgdat,line],
                      "layout": go.Layout(title = "Polynomial fit when lambda = 1 (just right)",
                                          xaxis = dict(title="Change in water level (x)"),
                                          yaxis = dict(title="Water flowing out of the dam (y)"))})

         Current function value: 12.488335
         Iterations: 7
         Function evaluations: 117
         Gradient evaluations: 105


In [19]:
polytrainerr, polycverr = learning_curve(add_bias_term(Xpoly),y,add_bias_term(Xvalpoly),yval,lda_init)
numtrngegs = list(range(1,13))
polytrainlearncurve = go.Scatter(x=numtrngegs, y=polytrainerr, mode="line", name = "Train")
polycvlearncurve = go.Scatter(x=numtrngegs, y=polycverr, mode="line", name="Cross Validation")
plotly.offline.iplot({"data": [polytrainlearncurve,polycvlearncurve],
                      "layout": go.Layout(title = "Learning curve for polynomial regression when lambda = 1 (just right)",
                                          xaxis = dict(title="Number of training examples used to learn the parameters of our model"),
                                          yaxis = dict(title="Error"))})

### For $\lambda = 100$, the model is bad, underfitting results. 

It fits the training data poorly and performs poorly on the cross-validation data.

In [20]:
lda_init = 100
theta_optimized = train_linear_regression(add_bias_term(Xpoly), y, lda_init)
yspan = add_bias_term(spanpoly)@theta_optimized
trgdat = go.Scatter(x=X, y=y, mode="markers", name = "Data")
line = go.Scatter(x=span, y=yspan, mode="line", name="Regression Curve")
plotly.offline.iplot({"data": [trgdat,line],
                      "layout": go.Layout(title = "Polynomial fit when lambda=100 (underfitting)",
                                          xaxis = dict(title="Change in water level (x)"),
                                          yaxis = dict(title="Water flowing out of the dam (y)"))})

         Current function value: 123.787166
         Iterations: 1
         Function evaluations: 50
         Gradient evaluations: 40


In [21]:
polytrainerr, polycverr = learning_curve(add_bias_term(Xpoly),y,add_bias_term(Xvalpoly),yval,lda_init)
numtrngegs = list(range(1,13))
polytrainlearncurve = go.Scatter(x=numtrngegs, y=polytrainerr, mode="line", name = "Train")
polycvlearncurve = go.Scatter(x=numtrngegs, y=polycverr, mode="line", name="Cross Validation")
plotly.offline.iplot({"data": [polytrainlearncurve,polycvlearncurve],
                      "layout": go.Layout(title = "Learning curve for polynomial regression when lambda = 100 (underfitting)",
                                          xaxis = dict(title="Number of training examples used to learn the parameters of our model"),
                                          yaxis = dict(title="Error"))})

## Selecting $\lambda$ using a cross validation set

As different values of $\lambda$ can also affect cross validation set performance, and hence the generalizability of our model, we also want to systematically search for optimal values of $\lambda$. In the machine learning community, $\lambda$ is called a **hyperparameter**, as its specific chosen value can constrain or determine what values we learn for our parameters $\theta_1 \dots \theta_n$ during the training procedure.

The curves that we draw to compare training and cross-validation errors for varying values of $\lambda$ are called **validation curves**. The function `validation_curve()` is similar to `learning_curve()`. 

TODO: clarify why I also do not regularize the calculation of the errors here?

In [22]:
def validation_curve(_X, _y, _Xval, _yval, _ldas):
    _trainerr = np.zeros_like(_ldas)
    _cverr  = np.zeros_like(_ldas)
    _thetainit = np.zeros(_X.shape[1])
    for i in range(0,len(_ldas)):
        _thetalearnt = fmin_cg(linear_regression_cost, _thetainit,
                               linear_regression_gradient, (_X, _y, _ldas[i]), disp=False)
        _trainerr[i] = linear_regression_cost(_thetalearnt, _X, _y, 0) #do not regularize the errors?
        _cverr[i] = linear_regression_cost(_thetalearnt, _Xval, _yval, 0)
    return _trainerr, _cverr

In [23]:
#lambdas = [0,0.001,0.003,0.01,0.03,0.1,0.3,1,1.5,2,2.5,3,3.5,4,5,6,7,8,9,10]
lambdas = [0,0.001,0.003,0.01,0.03,0.1,0.3,1,3,10]
valcurvetrgerr, valcurvecverr = validation_curve(add_bias_term(Xpoly),y,add_bias_term(Xvalpoly),yval,lambdas)

polytrainlearncurve = go.Scatter(x=lambdas, y=valcurvetrgerr, mode="line", name = "Train")
polycvlearncurve = go.Scatter(x=lambdas, y=valcurvecverr, mode="line", name="Cross Validation")
plotly.offline.iplot({"data": [polytrainlearncurve,polycvlearncurve],
                      "layout": go.Layout(title = "Validation curves for polynomial regression to select lambda",
                                          xaxis = dict(title="lambda"),
                                          yaxis = dict(title="Error"))})

We observ that the best value of $\lambda$ is at $3$, because that is the value that **minimizes the cross-validation error**.

TODO: The trends match those of figure 9 in ex5.pdf, but the actual values of the points on these curves do not. Find out why...

TODO: Need to clarify what happens when I insert more lambda values in between... from 5,6,7,8,9.... I would see more details and thus a somewhat different curve....would that affect my decision on 3 though?? For example using `lambdas =[0,0.001,0.003,0.01,0.03,0.1,0.3,1,1.5,2,2.5,3,3.5,4,5,6,7,8,9,10]`, I see that the best result is actually at lambda=2!

## Computing Test Set Error

To get a better indication of the model’s performance in the real world, it is important to evaluate the “final” model on a test set that was not used in any part of training (that is, it was neither used to select the λ parameters, nor to learn the model parameters θ).

TODO: Figure out why even though I followed all the steps here should be correct, I got such a different test error as the assignment PDF... unexpected value of 4.33 when it should've ben 3.8599...

In [24]:
lda_init = 3
theta_optimized = train_linear_regression(add_bias_term(Xpoly), y, lda_init)
theta_optimized

         Current function value: 31.721688
         Iterations: 5
         Function evaluations: 90
         Gradient evaluations: 78


array([ 11.22065879,   6.10725898,   3.57159262,   3.86964078,
         2.29166079,   2.37879646,   1.44963878,   1.43976211,   0.90654477])

In [25]:
# Compute feature scaling for test data, using the mu and sigma obtained from computing on the test data
Xtestpoly = poly_features(Xtest,deg)
Xtestpoly = Xtestpoly-mu
Xtestpoly = Xtestpoly / sigma
linear_regression_cost(theta_optimized, add_bias_term(Xtestpoly), ytest, 0)

4.3397328988063162

## Plotting learning curves with randomly selected examples

TODO: Explain the implementation here. The random selection of subsets of training examples was obtained through using a boolean mask and randomly permuting its rows. We then use this permuted boolean mask to select out rows randomly the array, thereby achieving random selection for a subset.

In [26]:
# numelm is the number of examples we want to select out with the mask, ie. number that controls
# the size of our subset
def generate_boolean_mask(_length, _numegs):
    _mask = np.zeros(_length, dtype=bool)
    for i in range(0, _numegs):
        _mask[i] = True
    return _mask

TODO: The number of rows (examples) of the training dataset is not equal to number of rows of the validation dataset, so do we still use the same mask `_length` for both training data and validation data, and index out an equal number of training examples from both? Must we also always index out the same random subset?

In [27]:
def learning_curve_with_rand_chosen_examples(_X, _y, _Xval, _yval, _lda, _numran):
    # Initialize variables for computation inputs and storage
    _trainerr = np.zeros_like(_y)
    _cverr = np.zeros_like(_y)
    numparams = _X.shape[1] # get the number of columns in our design matrix be equal to number of parameter needed to learn
    # Select out subset of our training data, starting from just first data point to eventuall all the data points
    for i in range(0, _y.size):
        count = 0
        errtrg = np.zeros(_numran) # accumulate the errors from random selection into a list
        errval = np.zeros(_numran)
        # prepare unshuffled boolean mask 
        mask = generate_boolean_mask(_X.shape[0], i+1)
        while(count<50):
            # Shuffle the mask at the start of each iteration, by randomly permuting the rows
            np.random.shuffle(mask)
            # masking out elements using the same randomly shuffled boolean mask has the effect of performing
            # random selection of a subset of i training examples
            _Xmasked = np.compress(mask, _X, axis=0)
            _ymasked = np.compress(mask, _y)
            _Xvalmasked = np.compress(mask, _Xval, axis=0)
            _yvalmasked = np.compress(mask, _yval)
            # Learn the parameters on the given subset of our training data, parameters initialized to 0 prior to training            
            _thetalearnt = fmin_cg(linear_regression_cost, np.zeros(numparams),
                                    linear_regression_gradient, (_Xmasked, _ymasked, _lda),  
                                    disp=False)
            # Compute the errors on the given subset of training and cross validation data
            # we accumulate 50 of errors into an array, values to be averaged out and stored later
            errtrg[count] = linear_regression_cost(_thetalearnt, _Xmasked, _ymasked, 0)
            errval[count] = linear_regression_cost(_thetalearnt, _Xvalmasked, _yvalmasked, 0)
            count = count+1
            
        _trainerr[i] = np.mean(errtrg)
        _cverr[i] = np.mean(errval)
        
    return _trainerr, _cverr

In [28]:
lda_init = 0.01
numran = 50 # Number of times to perform the random selection for a given size of subset of training examples
randtrainerr, randcverr = learning_curve_with_rand_chosen_examples(add_bias_term(Xpoly),y,
                                                                   add_bias_term(Xvalpoly),yval,lda_init, numran)

numtrngegs = list(range(1,13))
randtrainlearncurve = go.Scatter(x=numtrngegs, y=randtrainerr, mode="line", name = "Train")
randcvlearncurve = go.Scatter(x=numtrngegs, y=randcverr, mode="line", name="Cross Validation")
plotly.offline.iplot({"data": [randtrainlearncurve,randcvlearncurve],
                      "layout": go.Layout(title = "Polynomial regression learning curve with randomly selected examples, (lambda = 0.01)",
                                          xaxis = dict(title="Number of training examples used to learn the parameters of our model"),
                                          yaxis = dict(title="Error"))})

TODO: Trend is similar to figure 10 in ex5.pdf, but the actual values of the points on the curve are not, find out why...

# Testcases

WARNING: Messy WIP below....

Aside from the basic test values which we are told to compute in the assignment PDF itself, the course forums have test cases which are more rigorous. These testcases are executed here....

TODO: Wrap up all the below test cases into a single python module which will be imported directly into this notebook and executed in the cells below this one...

Cost function outputs match expected values.

TODO: Start to use formal unit testing constructs like `assert()` but they may not work too, because my values are not exact with the expected Octave output from the course skeleton code...

well this is a simple test case that was part of the assignment PDF, so I wonder if this should be wrapped in the module, or can it be a part of the work above (probably be a part of the work above...)

In [29]:
# Testing the cost function
X_bias = add_bias_term(X)
Theta = np.array([1,1])# Need to make Theta a column vector
linear_regression_cost(Theta, X_bias,y,1)

304.03485888693092

In [30]:
### TODO: Think of collating all the rigorous test cases from the course, and wrapping them up into a 
### single module that instantiates the TestCase() instance and runs unit tests to assert the truth of floating point
### outputs. Do not substitue these unit tests for places where I want to examine the outputs visually, as these are
### parts of the exercise where the correct output is given to us visually for us to manually inspect and compare 
### with the outputs of our own functions....

### See here below is an example...
#from unittest.TestCase import assertAlmostEqual
import unittest as utest
test = utest.TestCase()

test.assertAlmostEqual(first=linear_regression_cost(Theta, X_bias,y,1), second=303.993, delta = 0.5)

### Once I have coded out the module, just import it into this notebook, and add a chunk at the end where
### I subject all these functions to the unit tests just to show that they are correct. Purpose of this task is 
### to just get myself to pick up how to do unit testing.

#### Added on 28 July 2017
# Since coding out these unit tests are gonna take quite a lot of time, and I know that my functions here
# already correct, and its simple cases here since we only have 12 training examples, maybe I won't waste time here
# writing unit tests. But I want to still write these unit tests for practice so I will do it for Exercise 1, where 
# I've always suspected something was amiss with the regression on a plane.... may be wrong,so I can also implement
# gradient checking there, plus change the code to stop using numpy matrices and just use ndarrays with @ operator!

Gradient function outputs match the expected values.

In [31]:
linear_regression_gradient(Theta,X_bias,y,1)

array([ -15.30301567,  598.25074417])

Further testing cases
TODO: wrap this up into unit test module that I've written about earilier...

In [32]:
Xtestcase = np.array([[1,8,1,6],[1, 3,5 ,7],[1,4,9,2]])
ytestcase = np.array([7,6,5])
thetatest = np.array([0.1,0.2,0.3,0.4])
print(linear_regression_gradient(thetatest,Xtestcase,ytestcase,0)) # lamdba = 0
print(linear_regression_gradient(thetatest,Xtestcase,ytestcase,7)) # lamdba = 7
#test case results match the expected

[-1.4        -8.73333333 -4.33333333 -7.93333333]
[-1.4        -8.26666667 -3.63333333 -7.        ]


In [33]:
##More Test cases
xtc = np.array([1,2,3,4]); ytc=np.array([5]); thetatc = np.array([0.1,0.2,0.3,0.4])
print(linear_regression_cost(thetatc,xtc,ytc,7), "but expected", 3.015)
print(linear_regression_gradient(thetatc,xtc,ytc,7), "as expected")

3.05 but expected 3.015
[-2.  -2.6 -3.9 -5.2] as expected


In [34]:
# Test case for polynomial features
poly_features(np.array([1,2,3]),4)

array([[ 1,  1,  1,  1],
       [ 2,  4,  8, 16],
       [ 3,  9, 27, 81]])

In [35]:
# Test cases
Xtc = add_bias_term(np.array([2,3,4,5]))
Xvaltc = add_bias_term(np.array([7,-2]))
ytc=np.array([7,6,4,5])
yvaltc = np.array([2,12])
validation_curve(Xtc,ytc,Xvaltc,yvaltc,lambdas)
# results do not match expected

(array([ 0.225     ,  0.22639379,  0.24181128,  0.22577375,  0.23375395,
         0.42124037,  0.22619841,  1.99729238,  2.48294797,  3.31846602]),
 array([  1.225     ,   0.9195883 ,   0.35917267,   1.48273688,
          2.17472704,   8.46294276,   1.54312743,  42.82424024,
         48.99623317,  55.97614495]))

Below testcases... to move to a distinct python module and wrapped up with the assertion classes...

In [36]:
#tc stands for testcase
tcxval = np.array([  [-0.50000 ,  0.00000], [-0.40000  , 0.10000], [-0.30000,   0.20000],  
  [-0.20000,   0.30000],  [-0.10000,   0.40000],  [-0.50000  , 0.00000],
              [-0.40000 ,  0.10000],[-0.30000,   0.20000],[ -0.20000,   0.30000],[ -0.10000  , 0.40000]])
tcxval = add_bias_term(tcxval)
tcxval[:,0] = tcxval[:,0]/10.0
tcx= np.array([[-5.0,   0.0],
  [-4.0  , 1.0],
  [-3.0 ,  2.0],
  [-2.0 ,  3.0],
  [-1.0 ,  4.0]])
tcx = add_bias_term(tcx)

In [37]:
tcyval = np.array([ -0.20000,
  -0.10000,
   0.00000,
   0.10000,
   0.20000,
  -0.20000,
  -0.10000,
   0.00000,
   0.10000,
   0.20000])
tcy = np.array([
  -2.0,
  -1.0,
   0.0,
   1.0,
   2.0])

In [38]:
tcyval

array([-0.2, -0.1,  0. ,  0.1,  0.2, -0.2, -0.1,  0. ,  0.1,  0.2])

In [39]:
tctrerr , tccverr = learning_curve(_X=tcx,_y=tcy,_Xval=tcxval,_yval=tcyval,_lda=1)

In [40]:
# Why all 0s???
print(tctrerr)
print(tccverr)

[ 0.0029249   0.02647851  0.00056235  0.00069043  0.00246918]
[  1.09759270e-02   4.39773921e-03   6.53419752e-06   6.23719258e-06
   2.46917798e-05]
